## Injection benchmark

### Setup

In [1]:
import sys
from antidote import __version__, is_compiled
print(f"Antidote: {__version__()} {'(cython)' if is_compiled() else ''}")
print(f"Python {sys.version}")

Antidote: 0.8.0 (cython)
Python 3.8.5 (default, Aug 21 2020, 11:31:30) 
[GCC 9.3.0]


In [2]:
from antidote import world, Service, inject

class Service1(Service):
    pass


class Service2(Service):
    def __init__(self, service1: Service1):
        self.service1 = service1


class Service3(Service):
    def __init__(self, service1: Service1, service2: Service2):
        self.service1 = service1
        self.service2 = service2


class Service4(Service):
    def __init__(self, service1: Service1, service2: Service2, service3: Service3):
        self.service1 = service1
        self.service2 = service2
        self.service3 = service3

### Results
The key take away from those benchmarks, is to avoid using injection on short functions which are called repeatedly, in a loop typically. In the most common use case of dependency injection, service instantiation, the overhead should be negligible.

It should be noted that in most cases the worst scenario is used, as functions do nothing. In the real world, pure python functions are a lot slower. So to put the following results into perspective, here is the time needed to decode this simple JSON.

In [3]:
import json
%timeit json.loads('{ "name":"John", "age":30, "city":"New York"}')

5.55 µs ± 2.07 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Function call

Injection overhead is here measured with a function which does nothing.

In [4]:
def f(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    return s1, s2, s3, s4

Time necessary to only execute the function, without retrieving the services

In [5]:
args = (world.get(Service1), world.get(Service2), world.get(Service3), world.get(Service4))
%timeit f(*args)

181 ns ± 29.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Overhead of the injection when all argument must be retrieved from the container.

In [6]:
f_injected = inject(f)
assert f(*args) == f_injected()
%timeit f_injected()

729 ns ± 283 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Overhead of the injection when no argument has to be retrieved.

In [7]:
assert f(*args) == f_injected(*args)
%timeit f_injected(*args)

213 ns ± 7.39 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Method call

In [8]:
class Dummy:
    @inject
    def f(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        return s1, s2, s3, s4

In [9]:
%timeit Dummy().f(*args)

584 ns ± 34.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
d = Dummy()
assert d.f(*args) == d.f()
%timeit d.f()

832 ns ± 46.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Non singleton

We compare the overhead of creating the whole service each time.

In [11]:
class ServiceX(Service):
    __antidote__ = Service.Conf(singleton=False)

@inject
def f_not_singleton(s: ServiceX):
    return s

In [12]:
assert isinstance(f_not_singleton(), ServiceX)
assert f_not_singleton() is not f_not_singleton()
%timeit f_not_singleton()

651 ns ± 26.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Factory

In [13]:
from antidote import factory

class ServiceF:
    pass

@factory(singleton=False)
def service_factory() -> ServiceF:
    return ServiceF()

@inject(dependencies=(ServiceF @ service_factory,))
def f_factory(s):
    return s

In [14]:
assert isinstance(f_factory(), ServiceF)
assert f_factory() is not f_factory()
%timeit f_factory()

719 ns ± 44.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Custom Provider

In [15]:
import time
from antidote import world
from antidote.core import Provider, DependencyInstance
dep = object()
dep2 = object()

@world.provider
class SlowProvider(Provider):
    def exists(self, dependency):
        return dependency is dep2
    
    def provide(self, dependency, container):
        time.sleep(.01)
        return DependencyInstance("sleepy")
            

@world.provider
class CustomProvider(Provider):
    def exists(self, dependency):
        return dependency is dep
    
    def provide(self, dependency, container):
        return DependencyInstance("Found it !")


@inject(dependencies=dict(d=dep))
def f_provider(d):
    return d

In [16]:
assert f_provider() == "Found it !"
%timeit f_provider()

1.3 µs ± 77.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Object instantiation

In [17]:
class Obj:
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit Obj(*args)

509 ns ± 20.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [18]:
class ObjInjected:
    @inject
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit ObjInjected()

1.4 µs ± 48.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Configuration


In [19]:
from antidote import Constants

class Conf(Constants):
    A = 'A'
    B = 'B'

    def get(self, key):
        return key

In [20]:
def g(a, b):
    return a, b

%timeit g('A', 'B')

134 ns ± 7.48 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [21]:
conf = Conf()
%timeit g(conf.get('A'), conf.get('B'))

395 ns ± 14.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [22]:
g_injected = inject(g, dependencies=(Conf.A, Conf.B))

assert g(conf.get('A'), conf.get('B')) == g_injected()
assert g(conf.A, conf.B) == g_injected()

%timeit g_injected()

512 ns ± 21.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
